# Evaluation of RAG Using Ragas

In the following notebook we'll explore how to evaluate RAG pipelines using a powerful open-source tool called "Ragas". This will give us tools to evaluate component-wise metrics, as well as end-to-end metrics about the performance of our RAG pipelines.

In the following notebook we'll complete the following tasks:

- 🤝 Breakout Room #1:
  1. Install required libraries
  2. Set Environment Variables
  3. Creating a simple RAG pipeline with [LangChain v0.1.0](https://blog.langchain.dev/langchain-v0-1-0/)
  

- 🤝 Breakout Room #2:
  1. Synthetic Dataset Generation for Evaluation using the [Ragas](https://github.com/explodinggradients/ragas)
  2. Evaluating our pipeline with Ragas
  3. Making Adjustments to our RAG Pipeline
  4. Evaluating our Adjusted pipeline against our baseline
  5. Testing OpenAI's Claim

The only way to get started is to get started - so let's grab our dependencies for the day!

## Motivation

A claim, made by OpenAI, is that their `text-embedding-3-small` is better (generally) than their `text-embedding-ada-002` model.

Here's some passages from their [blog](https://openai.com/blog/new-embedding-models-and-api-updates) about the `text-embedding-3` release:

> `text-embedding-3-small` is our new highly efficient embedding model and provides a significant upgrade over its predecessor, the `text-embedding-ada-002` model...

> **Stronger performance.** Comparing `text-embedding-ada-002` to `text-embedding-3-small`, the average score on a commonly used benchmark for multi-language retrieval ([MIRACL](https://github.com/project-miracl/miracl)) has increased from 31.4% to 44.0%, while the average score on a commonly used benchmark for English tasks ([MTEB](https://github.com/embeddings-benchmark/mteb)) has increased from 61.0% to 62.3%.

Well, with a library like Ragas - we can put that claim to the test!

If what they claim is true - we should see an increase on related metrics by using the new embedding model!

# 🤝 Breakout Room #1

## Task 1: Installing Required Libraries

A reminder that one of the [key features](https://blog.langchain.dev/langchain-v0-1-0/) of LangChain v0.1.0 is the compartmentalization of the various LangChain ecosystem packages!

So let's begin grabbing all of our LangChain related packages!

In [1]:
!pip install -U -q langchain langchain-openai langchain_core langchain-community langchainhub openai

We'll also get the "star of the show" today, which is Ragas!

In [2]:
!pip install -qU ragas

As well, instead of the remote hosted solution that we used last week (Pinecone), we'll be leveraging Meta's [FAISS](https://github.com/facebookresearch/faiss) as the backend for our LangChain `VectorStore`.

We'll also install `unstructured` (from [Unstructured-IO](https://github.com/Unstructured-IO/unstructured)) and its dependencies which will allow us to load PDFs using the `UnstructuredPDFLoader` in the `langchain-community` package!

In [3]:
!pip install -qU faiss_cpu pymupdf pandas

## Task 2: Set Environment Variables

Let's set up our OpenAI API key so we can leverage their API later on.

In [4]:
import os
import openai
from getpass import getpass

openai.api_key = getpass("Please provide your OpenAI Key: ")
os.environ["OPENAI_API_KEY"] = openai.api_key

## Task 3: Creating a Simple RAG Pipeline with LangChain v0.1.0

Building on what we learned last week, we'll be leveraging LangChain v0.1.0 and LCEL to build a simple RAG pipeline that we can baseline with Ragas.

## Building our RAG pipeline

Let's review the basic steps of RAG again:

- Create an Index
- Use retrieval to obtain pieces of context from our Index that are similar to our query
- Use a LLM to generate responses based on the retrieved context

Let's get started by creating our index.

> NOTE: We're going to start leaning on the term "index" to refer to our `VectorStore`, `VectorDatabase`, etc. We can think of "index" as the catch-all term, whereas `VectorStore` and the like relate to the specific technologies used to create, store, and interact with the index.

### Creating an Index

You'll notice that the largest changes (outside of some import changes) are that our old favourite chains are back to being bundled in an easily usable abstraction.

We can still create custom chains using LCEL - but we can also be more confident that our pre-packaged chains are creating using LCEL under the hood.

#### Loading Data

Let's start by loading some data!

> NOTE: You'll notice that we're using a document loader from the community package of LangChain. This is part of the v0.1.0 changes that make the base (`langchain-core`) package remain lightweight while still providing access to some of the more powerful community integrations.

In [5]:
!git clone https://github.com/AI-Maker-Space/DataRepository

fatal: destination path 'DataRepository' already exists and is not an empty directory.


In [6]:
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader(
    "DataRepository/MuskComplaint.pdf",
)

documents = loader.load()

In [7]:
documents[0].metadata

{'source': 'DataRepository/MuskComplaint.pdf',
 'file_path': 'DataRepository/MuskComplaint.pdf',
 'page': 0,
 'total_pages': 46,
 'format': 'PDF 1.7',
 'title': '',
 'author': '',
 'subject': '',
 'keywords': '',
 'creator': '',
 'producer': '',
 'creationDate': '',
 'modDate': '',
 'trapped': ''}

#### Transforming Data

Now that we've got our single document - let's split it into smaller pieces so we can more effectively leverage it with our retrieval chain!

We'll start with the classic: `RecursiveCharacterTextSplitter`.

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 700,
    chunk_overlap = 50
)

documents = text_splitter.split_documents(documents)

Let's confirm we've split our document.

In [9]:
len(documents)

159

#### Loading OpenAI Embeddings Model

We'll need a process by which we can convert our text into vectors that allow us to compare to our query vector.

Let's use OpenAI's `text-embedding-ada-002` for this task!

In [10]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002"
)

#### Creating a FAISS VectorStore

Now that we have documents - we'll need a place to store them alongside their embeddings.

In [11]:
from langchain_community.vectorstores import FAISS

vector_store = FAISS.from_documents(documents, embeddings)

####❓ Question #1:

List out a few of the techniques that FAISS uses that make it performant.

* Makes using of multi-threading to exploit multiple cores and perform parallel searches (when GPUs are available)
* BLAS libraries for efficient exact distance computations via matrix/matrix multiplication
* Machine SIMD vectorization and popcount are used to speed up distance computations for isolated vectors

#### Creating a Retriever

To complete our index, all that's left to do is expose our vectorstore as a retriever - which we can do the same way we would in previous version of LangChain!

In [76]:
retriever = vector_store.as_retriever()

#### Testing our Retriever

Now that we've gone through the trouble of creating our retriever - let's see it in action!

In [77]:
retrieved_documents = retriever.invoke("Who is the plantiff?")

In [14]:
for doc in retrieved_documents:
  print(doc)

page_content='would be owned by the foundation and used ‘for the good of the world’[.]” Plaintiff \nreplied: “Agree on all.” Ex. 2 at 1.' metadata={'source': 'DataRepository/MuskComplaint.pdf', 'file_path': 'DataRepository/MuskComplaint.pdf', 'page': 27, 'total_pages': 46, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': '', 'creationDate': '', 'modDate': '', 'trapped': ''}
page_content='property and derivative works funded by those monies, Plaintiff is presently unable to ascertain his \ninterest in or the use, allocation, or distribution of assets without an accounting. Plaintiff is therefore \nentitled to an accounting.' metadata={'source': 'DataRepository/MuskComplaint.pdf', 'file_path': 'DataRepository/MuskComplaint.pdf', 'page': 32, 'total_pages': 46, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': '', 'creationDate': '', 'modDate': '', 'trapped': ''}
page_content='1

### Creating a RAG Chain

Now that we have the "R" in RAG taken care of - let's look at creating the "AG"!

#### Creating a Prompt Template

There are a few different ways we could create our prompt template - we could create a custom template, as seen in the code below, or we could simply pull a prompt from the prompt hub! Let's look at an example of that!

In [15]:
from langchain import hub

retrieval_qa_prompt = hub.pull("langchain-ai/retrieval-qa-chat")

In [16]:
print(retrieval_qa_prompt.messages[0].prompt.template)

Answer any use questions based solely on the context below:

<context>
{context}
</context>


As you can see - the prompt template is simple (and has a small error) - so we'll create our own to be a bit more specific!

In [17]:
from langchain.prompts import ChatPromptTemplate

template = """Answer the question based only on the following context. If you cannot answer the question with the context, please respond with 'I don't know':

Context:
{context}

Question:
{question}
"""

prompt = ChatPromptTemplate.from_template(template)

#### Setting Up our Basic QA Chain

Now we can instantiate our basic RAG chain!

We'll use LCEL directly just to see an example of it - but you could just as easily use an abstraction here to achieve the same goal!

We'll also ensure to pass-through our context - which is critical for RAGAS.

In [25]:
from operator import itemgetter

from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

primary_qa_llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

retrieval_augmented_qa_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough()
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": prompt | primary_qa_llm, "context": itemgetter("context")}
)

####🏗️ Activity #1:

Describe the pipeline shown above in simple terms. You can include a diagram if desired.

1. User query is passed in as a single item dictionary with key "question"
2. A new dictionary is created with two keys.
    * "context": populated by retrieving "question" value from input dictionary and chaining it into the retreiver
    * "question" populated by retreiving "question" value from the input dictionary
3. We create a runnable pass through of the context and question (although only the context is explicitely set, the question is implicitely passed) - the runnable pass through is then consumable by the next runnable which will be used to populate the "prompt"
4. Finally we populate a new dictionary with two keys:
    * "response": populate by running the chain prompt | primary_qa_llm - where the context and question for the prompt come from the runnable pass through
    * "context": populated by retreiving "context" value from the input dictionary

In [ ]:
retrieval_augmented_qa_chain = (
    RunnablePassthrough.assign(context = (itemgetter("question") | retriever), question = itemgetter("question"))
    | {"response": prompt | primary_qa_llm, "context": itemgetter("context")}
)

Let's test it out!

In [34]:
question = "Who is the plantiff?"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result["response"].content)

Elon Musk


In [36]:
question = "What does this complaint pertain to?"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result["response"].content)

The complaint pertains to breach of fiduciary duty, unfair business practices, and accounting.


We can already see that there are some improvements we could make here.

For now, let's switch gears to RAGAS to see how we can leverage that tool to provide us insight into how our pipeline is performing!

# 🤝 Breakout Room #2

## Task 1: Synthetic Dataset Generation for Evaluation using Ragas

Ragas is a powerful library that lets us evaluate our RAG pipeline by collecting input/output/context triplets and obtaining metrics relating to a number of different aspects of our RAG pipeline.

We'll be evluating on every core metric today, but in order to do that - we'll need to creat a test set. Luckily for us, Ragas can do that directly!

### Synthetic Test Set Generation

We can leverage Ragas' [`Synthetic Test Data generation`](https://docs.ragas.io/en/stable/concepts/testset_generation.html) functionality to generate our own synthetic QC pairs - as well as a synthetic ground truth - quite easily!

> NOTE: This process will use `gpt-3.5-turbo-16k` as the base generator and `gpt-4` as the critic - if you're attempting to create a lot of samples please be aware of cost, as well as rate limits.

In [37]:
eval_documents = documents

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 400
)

eval_documents = text_splitter.split_documents(eval_documents)

####❓ Question #2:

Why is it important to split our documents using different parameters when creating our synthetic data?

Answer: It is important because if we use the same parameters, then we won't get realistic measurements of performance using RAGAS. Focusing on "simple" questions for a moment. In real life, we don't know what our "simple" questions are actually going to be. We want our synthetic questions to test our RAG system in a "realistic" way. If we use the same chunking, then all of our "simple" questions are going to be answerable from a single chunk in our vector store - but this makes the job a) easier for our RAG system to answer the question correctly and b) makes it such that our eval. metrics are going to be inflated compared to reality. (which will be true of the more complex questions too).

159

In [38]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context

generator = TestsetGenerator.with_openai()

testset = generator.generate_with_langchain_docs(documents, test_size=10, distributions={simple: 0.25, reasoning: 0.25, multi_context: 0.5})

/home/smonk/.conda/envs/week4_day1/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Filename and doc_id are the same for all nodes.                   
Generating: 100%|██████████| 10/10 [00:26<00:00,  2.67s/it]


####❓ Question #3:

`{simple: 0.5, reasoning: 0.25, multi_context: 0.25}`

What exactly does this mapping refer to?

> NOTE: Check out the Ragas documentation on this generation process [here](https://docs.ragas.io/en/stable/concepts/testset_generation.html).

Answer:
1. Simple: 50 % - questions will be derived from single chunks
2. Reasoning: 25% - Rewrites the question in a way that enhances the need for reasoning to answer effectively
3. Multi_context: 25% - Rephrase the question in a manner that necessitates information from multiple related sections or chunks to formulate an answer.

Let's look at the output and see what we can learn about it!

In [ ]:
testset.test_data[0]

DataRow(question="What did Mr. Nadella state during the interview regarding OpenAI's potential disappearance?", contexts=['Indeed, during an interview shortly after Mr. Altman’s firing, Mr. Nadella stated: \n[W]e were very confident in our own ability. We have all the IP rights and all the \ncapability. If OpenAI disappeared tomorrow, I don’t want any customer of ours to \nbe worried about it quite honestly, because we have all of the rights to continue the \ninnovation. Not just to serve the product, but we can go and just do what we were \ndoing in partnership ourselves. We have the people, we have the compute, we have \nthe data, we have everything.'], ground_truth="Mr. Nadella stated that if OpenAI disappeared tomorrow, he doesn't want any customer to be worried because Microsoft has all the rights to continue the innovation.", evolution_type='simple')

### Generating Responses with RAG Pipeline

Now that we have some QC pairs, and some ground truths, let's evaluate our RAG pipeline using Ragas.

The process is, again, quite straightforward - thanks to Ragas and LangChain!

Let's start by extracting our questions and ground truths from our create testset.

We can start by converting our test dataset into a Pandas DataFrame.

In [39]:
test_df = testset.to_pandas()

In [40]:
test_df

,question,contexts,ground_truth,evolution_type,episode_done
0,What is the approach of OpenAI towards the dev...,[profit developing AGI for the benefit of huma...,OpenAI's approach towards the development of A...,simple,True
1,What game did OpenAI compete in using reinforc...,[77. \nInitial work at OpenAI followed much in...,OpenAI competed in Dota 2 using reinforcement ...,simple,True
2,What sets large language models apart from pre...,[which needed to be trained on a specific task...,Large language models are trained on a suffici...,reasoning,True
3,"What has OpenAI abandoned for profit, as state...",[process” revealed that resulted in the initia...,OpenAI has abandoned its 'irrevocable' non-pro...,reasoning,True
4,"""What technique showcased the reasoning abilit...",[implementation for others to build on. \n84. ...,chain-of-thought prompting,multi_context,True
5,"""What game did OpenAI use reinforcement learni...",[77. \nInitial work at OpenAI followed much in...,OpenAI used reinforcement learning to beat a w...,multi_context,True
6,What assurance did Mr. Nadella provide regardi...,"[Indeed, during an interview shortly after Mr....",Mr. Nadella assured that if OpenAI disappeared...,multi_context,True
7,"""What algorithm does the Transformer use to tr...",[1 \n2 \n3 \n4 \n5 \n6 \n7 \n8 \n9 \n10 \n11 \...,The Transformer algorithm is used to translate...,multi_context,True
8,What were Stephen Hawking's concerns about AGI...,[18. \nMr. Musk has long recognized that AGI p...,Stephen Hawking's concerns about AGI in the wr...,multi_context,True
9,How did OpenAI use a deep neural network in th...,[those connections to the target language. \n7...,By using the first half of Google's Transforme...,simple,True


In [41]:
test_questions = test_df["question"].values.tolist()
test_groundtruths = test_df["ground_truth"].values.tolist()

Now we'll generate responses using our RAG pipeline using the questions we've generated - we'll also need to collect our retrieved contexts for each question.

We'll do this in a simple loop to see exactly what's happening!

In [42]:
answers = []
contexts = []

for question in test_questions:
  response = retrieval_augmented_qa_chain.invoke({"question" : question})
  answers.append(response["response"].content)
  contexts.append([context.page_content for context in response["context"]])

Now we can wrap our information in a Hugging Face dataset for use in the Ragas library.

In [43]:
from datasets import Dataset

response_dataset = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

Let's take a peek and see what that looks like!

In [44]:
response_dataset[0]

{'question': 'What is the approach of OpenAI towards the development of AGI?',
 'answer': "OpenAI's approach towards the development of AGI is focused on developing AGI for the benefit of humanity, not for a for-profit company seeking to maximize shareholder profits. They aim to keep their technology open-source and not keep it closed and secret for proprietary commercial reasons.",
 'contexts': ['1 \n2 \n3 \n4 \n5 \n6 \n7 \n8 \n9 \n10 \n11 \n12 \n13 \n14 \n15 \n16 \n17 \n18 \n19 \n20 \n21 \n22 \n23 \n24 \n25 \n26 \n27 \n28 \n \n \n– 5 – \nCOMPLAINT \n \npublicly called for a variety of measures to address the dangers of AGI, from voluntary moratoria \nto regulation, but his calls largely fell on deaf ears.  \n19. \nBut where some like Mr. Musk see an existential threat in AGI, others see AGI as a \nsource of profit and power.  \n20. \nIn 2014, Google acquired DeepMind, a research group focused on deep learning. \nOne of DeepMind’s initial developments was AlphaZero, a chess playing al

## Task 2: Evaluating our Pipeline with Ragas

Now that we have our response dataset - we can finally get into the "meat" of Ragas - evaluation!

First, we'll import the desired metrics, then we can use them to evaluate our created dataset!

Check out the specific metrics we'll be using in the Ragas documentation:

- [Faithfulness](https://docs.ragas.io/en/stable/concepts/metrics/faithfulness.html)
- [Answer Relevancy](https://docs.ragas.io/en/stable/concepts/metrics/answer_relevance.html)
- [Context Precision](https://docs.ragas.io/en/stable/concepts/metrics/context_precision.html)
- [Context Recall](https://docs.ragas.io/en/stable/concepts/metrics/context_recall.html)
- [Answer Correctness](https://docs.ragas.io/en/stable/concepts/metrics/answer_correctness.html)

See the accompanied presentation for more in-depth explanations about each of the metrics!

In [45]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    answer_correctness,
    context_recall,
    context_precision,
)

metrics = [
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    answer_correctness,
]

All that's left to do is call "evaluate" and away we go!

In [46]:
results = evaluate(response_dataset, metrics)

Evaluating: 100%|██████████| 50/50 [00:13<00:00,  3.65it/s]


In [47]:
results

{'faithfulness': 0.9000, 'answer_relevancy': 0.8485, 'context_recall': 0.9667, 'context_precision': 0.8167, 'answer_correctness': 0.8080}

In [48]:
results_df = results.to_pandas()
results_df

,question,answer,contexts,ground_truth,faithfulness,answer_relevancy,context_recall,context_precision,answer_correctness
0,What is the approach of OpenAI towards the dev...,OpenAI's approach towards the development of A...,[1 \n2 \n3 \n4 \n5 \n6 \n7 \n8 \n9 \n10 \n11 \...,OpenAI's approach towards the development of A...,1.0,0.993964,1.000000,0.500000,0.745302
1,What game did OpenAI compete in using reinforc...,Dota 2,[77. \nInitial work at OpenAI followed much in...,OpenAI competed in Dota 2 using reinforcement ...,1.0,0.988038,1.000000,1.000000,0.962706
2,What sets large language models apart from pre...,Large language models are different from previ...,[which needed to be trained on a specific task...,Large language models are trained on a suffici...,1.0,0.957859,1.000000,0.750000,0.613831
3,"What has OpenAI abandoned for profit, as state...","OpenAI has abandoned its ""irrevocable"" non-pro...",[“be explicit that I have no financial interes...,OpenAI has abandoned its 'irrevocable' non-pro...,1.0,0.000000,1.000000,0.500000,0.992412
4,"""What technique showcased the reasoning abilit...",Chain-of-thought prompting,[implementation for others to build on. \n84. ...,chain-of-thought prompting,1.0,0.869085,1.000000,0.916667,0.996954
5,"""What game did OpenAI use reinforcement learni...",Dota 2,[77. \nInitial work at OpenAI followed much in...,OpenAI used reinforcement learning to beat a w...,0.0,0.926096,0.666667,0.750000,0.961317
6,What assurance did Mr. Nadella provide regardi...,"Mr. Nadella stated that if OpenAI disappeared,...","[Indeed, during an interview shortly after Mr....",Mr. Nadella assured that if OpenAI disappeared...,1.0,0.942198,1.000000,1.000000,0.617075
7,"""What algorithm does the Transformer use to tr...",The Transformer algorithm uses connections bet...,[those connections to the target language. \n7...,The Transformer algorithm is used to translate...,1.0,0.885777,1.000000,0.750000,0.727521
8,What were Stephen Hawking's concerns about AGI...,Stephen Hawking's concerns about AGI in the wr...,[18. \nMr. Musk has long recognized that AGI p...,Stephen Hawking's concerns about AGI in the wr...,1.0,0.964590,1.000000,1.000000,0.724574
9,How did OpenAI use a deep neural network in th...,OpenAI used the first half of Google's Transfo...,[those connections to the target language. \n7...,By using the first half of Google's Transforme...,1.0,0.957506,1.000000,1.000000,0.738721


## Task 3: Making Adjustments to our RAG Pipeline

Now that we have established a baseline - we can see how any changes impact our pipeline's performance!

Let's modify our retriever and see how that impacts our Ragas metrics!

In [49]:
from langchain.retrievers import MultiQueryRetriever

advanced_retriever = MultiQueryRetriever.from_llm(retriever=retriever, llm=primary_qa_llm)

We'll also re-create our RAG pipeline using the abstractions that come packaged with LangChain v0.1.0!

First, let's create a chain to "stuff" our documents into our context!

In [50]:
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(primary_qa_llm, retrieval_qa_prompt)

Next, we'll create the retrieval chain!

In [51]:
from langchain.chains import create_retrieval_chain

retrieval_chain = create_retrieval_chain(advanced_retriever, document_chain)

In [52]:
response = retrieval_chain.invoke({"input": "Who is the plantiff?"})

In [53]:
print(response["answer"])

The plaintiff is Elon Musk.


In [54]:
response = retrieval_chain.invoke({"input": "What does this complaint pertain to?"})

In [55]:
print(response["answer"])

This complaint pertains to a legal case involving Plaintiff Elon Musk alleging breach of fiduciary duty, unfair business practices, and seeking an accounting from the Defendants. The complaint also includes a demand for a jury trial.


Well, just from those responses this chain *feels* better - but lets see how it performs on our eval!

Let's do the same process we did before to collect our pipeline's contexts and answers.

In [56]:
answers = []
contexts = []

for question in test_questions:
  response = retrieval_chain.invoke({"input" : question})
  answers.append(response["answer"])
  contexts.append([context.page_content for context in response["context"]])

Now we can convert this into a dataset, just like we did before.

In [57]:
response_dataset_advanced_retrieval = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

Let's evaluate on the same metrics we did for the first pipeline and see how it does!

In [58]:
advanced_retrieval_results = evaluate(response_dataset_advanced_retrieval, metrics)

Evaluating:   0%|          | 0/50 [00:00<?, ?it/s]

Evaluating: 100%|██████████| 50/50 [00:37<00:00,  1.35it/s]


In [59]:
advanced_retrieval_results_df = advanced_retrieval_results.to_pandas()
advanced_retrieval_results_df

,question,answer,contexts,ground_truth,faithfulness,answer_relevancy,context_recall,context_precision,answer_correctness
0,What is the approach of OpenAI towards the dev...,OpenAI's approach towards the development of A...,[1 \n2 \n3 \n4 \n5 \n6 \n7 \n8 \n9 \n10 \n11 \...,OpenAI's approach towards the development of A...,NaN,0.993964,1.00,0.337302,0.671418
1,What game did OpenAI compete in using reinforc...,"OpenAI competed in Dota 2, a strategy video ga...",[77. \nInitial work at OpenAI followed much in...,OpenAI competed in Dota 2 using reinforcement ...,1.0,1.000000,1.00,1.000000,0.746921
2,What sets large language models apart from pre...,Large language models are different from previ...,[which needed to be trained on a specific task...,Large language models are trained on a suffici...,1.0,0.957859,1.00,0.750000,0.667133
3,"What has OpenAI abandoned for profit, as state...","OpenAI has abandoned its ""irrevocable"" non-pro...",[1 \n2 \n3 \n4 \n5 \n6 \n7 \n8 \n9 \n10 \n11 \...,OpenAI has abandoned its 'irrevocable' non-pro...,1.0,0.000000,1.00,0.816667,0.741518
4,"""What technique showcased the reasoning abilit...",The technique that showcased the reasoning abi...,[implementation for others to build on. \n84. ...,chain-of-thought prompting,1.0,0.913904,1.00,0.950000,0.964533
5,"""What game did OpenAI use reinforcement learni...",OpenAI used reinforcement learning to beat a w...,[77. \nInitial work at OpenAI followed much in...,OpenAI used reinforcement learning to beat a w...,1.0,0.894045,0.75,0.833333,0.615072
6,What assurance did Mr. Nadella provide regardi...,"Mr. Nadella stated that if OpenAI disappeared,...","[Indeed, during an interview shortly after Mr....",Mr. Nadella assured that if OpenAI disappeared...,1.0,0.942198,1.00,1.000000,0.618383
7,"""What algorithm does the Transformer use to tr...","The Transformer algorithm, initially developed...",[those connections to the target language. \n7...,The Transformer algorithm is used to translate...,0.8,0.910161,1.00,0.700000,0.742953
8,What were Stephen Hawking's concerns about AGI...,"Stephen Hawking, along with other luminaries l...",[18. \nMr. Musk has long recognized that AGI p...,Stephen Hawking's concerns about AGI in the wr...,1.0,0.924122,1.00,1.000000,0.528420
9,How did OpenAI use a deep neural network in th...,OpenAI used a deep neural network by pre-train...,[those connections to the target language. \n7...,By using the first half of Google's Transforme...,1.0,0.836574,1.00,1.000000,0.834551


## Task 4: Evaluating our Adjusted Pipeline Against Our Baseline

Now we can compare our results and see what directional changes occured!

Let's refresh with our initial metrics.

In [60]:
results

{'faithfulness': 0.9000, 'answer_relevancy': 0.8485, 'context_recall': 0.9667, 'context_precision': 0.8167, 'answer_correctness': 0.8080}

And see how our advanced retrieval modified our chain!

In [61]:
advanced_retrieval_results

{'faithfulness': 0.9778, 'answer_relevancy': 0.8373, 'context_recall': 0.9750, 'context_precision': 0.8387, 'answer_correctness': 0.7131}

In [62]:
import pandas as pd

df_original = pd.DataFrame(list(results.items()), columns=['Metric', 'Baseline'])
df_comparison = pd.DataFrame(list(advanced_retrieval_results.items()), columns=['Metric', 'MultiQueryRetriever with Document Stuffing'])

df_merged = pd.merge(df_original, df_comparison, on='Metric')

df_merged['Delta'] = df_merged['MultiQueryRetriever with Document Stuffing'] - df_merged['Baseline']

df_merged

,Metric,Baseline,MultiQueryRetriever with Document Stuffing,Delta
0,faithfulness,0.900000,0.977778,0.077778
1,answer_relevancy,0.848511,0.837283,-0.011229
2,context_recall,0.966667,0.975000,0.008333
3,context_precision,0.816667,0.838730,0.022063
4,answer_correctness,0.808041,0.713090,-0.094951


## Task 5: Testing OpenAI's Claim

Now that we've seen how our retriever can impact the performance of our RAG pipeline - let's see how changing our embedding model impacts performance.

####🏗️ Activity #2:

Please provide markdown, or code comments, to explain which each of the following steps are doing!

In [63]:
# Instantiate the Embeddings model (a new, theoretically better model from Open AI)
new_embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [64]:
# Create a new vector store that contains the embeddings from our chunked documents using our new embeddings model
vector_store = FAISS.from_documents(documents, new_embeddings)

In [66]:
# Instantiate our vector store as a retriever we can use in a chain
new_retriever = vector_store.as_retriever()

In [69]:
# Instantiates a retriever that will convert a single user query into one that generates multiple queries from different perspectives
# For each of those queries, it will retrieve relevant documents and uninion the results to provide to our context window
new_advanced_retriever = MultiQueryRetriever.from_llm(retriever=new_retriever, llm=primary_qa_llm)

In [70]:
# Instantiate a new retrieval chain that will use our new advanced retriever and the document stuffing chain
new_retrieval_chain = create_retrieval_chain(new_advanced_retriever, document_chain)

In [71]:
answers = []
contexts = []

for question in test_questions:
  response = new_retrieval_chain.invoke({"input" : question})
  answers.append(response["answer"])
  contexts.append([context.page_content for context in response["context"]])

In [72]:
new_response_dataset_advanced_retrieval = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

In [73]:
new_advanced_retrieval_results = evaluate(new_response_dataset_advanced_retrieval, metrics)

Evaluating:   0%|          | 0/50 [00:00<?, ?it/s]

Evaluating: 100%|██████████| 50/50 [00:22<00:00,  2.21it/s]


In [74]:
new_advanced_retrieval_results

{'faithfulness': 1.0000, 'answer_relevancy': 0.9314, 'context_recall': 0.9500, 'context_precision': 0.8760, 'answer_correctness': 0.7461}

In [75]:
df_baseline = pd.DataFrame(list(results.items()), columns=['Metric', 'Baseline'])
df_original = pd.DataFrame(list(advanced_retrieval_results.items()), columns=['Metric', 'ADA'])
df_comparison = pd.DataFrame(list(new_advanced_retrieval_results.items()), columns=['Metric', 'Text Embedding 3'])

df_merged = pd.merge(df_original, df_comparison, on='Metric')
df_merged = pd.merge(df_baseline, df_merged, on="Metric")

df_merged['Delta - TE3 -> ADA'] = df_merged['Text Embedding 3'] - df_merged['ADA']
df_merged['Delta - TE3 -> Baseline'] = df_merged['Text Embedding 3'] - df_merged['Baseline']

df_merged

,Metric,Baseline,ADA,Text Embedding 3,Delta - TE3 -> ADA,Delta - TE3 -> Baseline
0,faithfulness,0.900000,0.977778,1.000000,0.022222,0.100000
1,answer_relevancy,0.848511,0.837283,0.931376,0.094093,0.082865
2,context_recall,0.966667,0.975000,0.950000,-0.025000,-0.016667
3,context_precision,0.816667,0.838730,0.875952,0.037222,0.059286
4,answer_correctness,0.808041,0.713090,0.746070,0.032980,-0.061971


####❓ Question #4:

Do you think, in your opinion, `text-embedding-3-small` is significantly better than `ada`?

## BONUS ACTIVITY: Showcase Multi-Context Perfomance Changes

Now that we've looked at a number of different examples - showcase the difference on the multi-context *specific* questions that were synthetically generated.

> NOTE: You have all the data you'll need already in the notebook if you made it to this step!